In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.utils.data_utils import get_file
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import SimpleRNN, TimeDistributed
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv1D, MaxPooling1D, ZeroPadding1D
from keras.utils import np_utils
from keras.optimizers import Adam
#import cPickle as pickle
import pickle
import bcolz
import re
from numpy.random import random, permutation, randn, normal, uniform, choice
#C:\Users\dell\Desktop

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
text_file = open('star-wars.txt').read()
print(len(text_file))

145260


Finding number of unique characters in the text 

In [4]:
chars = sorted(list(set(text_file)))
print(len(chars)+1)

78


Converting unique characters into index

In [5]:
char_to_index = {v:i for i,v in enumerate(chars)}

Converting index into characters

In [6]:
index_to_char = {i:v for i,v in enumerate(chars)}

converting all the characters in the text into index

In [11]:
total_index = [char_to_index[char] for char in text_file]

In [12]:
len(total_index)

145260

Index of top ten characters

In [13]:
total_index[:10]

[25, 1, 62, 65, 64, 57, 1, 70, 59, 63]

In [14]:
''.join(index_to_char[i] for i in total_index[:41])

'A long time ago in a galaxy far, far away'

Input to RNN and expected output for training

In [15]:
pred_num = 7
xin = [[total_index[j+i] for j in range(0, len(total_index)-1-pred_num, pred_num)] for i in range(pred_num)]
y = [total_index[i+pred_num] for i in range(0, len(total_index)-1-pred_num, pred_num)]

In [16]:
X = [np.stack(xin[i][:-2]) for i in range(pred_num)]
Y = np.stack(y[:-2])

In [17]:
X

[array([25, 70, 65, ..., 64, 60, 57]),
 array([ 1, 59,  1, ..., 70, 71, 59]),
 array([62, 63, 59, ..., 71, 69, 64]),
 array([65, 55, 64, ..., 68, 70, 64]),
 array([64,  1,  1, ..., 55,  1, 59]),
 array([57, 51, 51, ...,  8, 52, 64]),
 array([ 1, 57,  1, ...,  1, 55, 57])]

In [18]:
Y[:8]

array([70, 65, 57, 56, 68, 10, 42,  1])

Defining parameters for model

In [73]:
hidden_layers = 256
vocab_size = 78
n_fac = 42

Model

In [74]:
model = Sequential([
        Embedding(vocab_size, n_fac, input_length=pred_num),
        SimpleRNN(hidden_layers, activation='relu'),
        Dense(vocab_size, activation='softmax')
    ])

In [75]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 7, 42)             3276      
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 256)               76544     
_________________________________________________________________
dense_4 (Dense)              (None, 78)                20046     
Total params: 99,866
Trainable params: 99,866
Non-trainable params: 0
_________________________________________________________________


Defining loss function

In [76]:
from keras import optimizers
#adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [77]:
model.fit(np.stack(X, 1), Y, batch_size=64, epochs=10)

Epoch 1/10
20749/20749 [==============================] - 3s 137us/step - loss: 3.3225
Epoch 2/10
20749/20749 [==============================] - 2s 111us/step - loss: 2.6759
Epoch 3/10
20749/20749 [==============================] - 2s 111us/step - loss: 2.4354
Epoch 4/10
20749/20749 [==============================] - 2s 109us/step - loss: 2.2779
Epoch 5/10
20749/20749 [==============================] - 2s 109us/step - loss: 2.1455
Epoch 6/10
20749/20749 [==============================] - 2s 109us/step - loss: 2.0302
Epoch 7/10
20749/20749 [==============================] - 2s 110us/step - loss: 1.9239
Epoch 8/10
20749/20749 [==============================] - 2s 109us/step - loss: 1.8272
Epoch 9/10
20749/20749 [==============================] - 2s 110us/step - loss: 1.7333
Epoch 10/10
20749/20749 [==============================] - 2s 109us/step - loss: 1.6471


In [78]:
model.save_weights('simpleRNN_3pred.h5')

In [63]:
model.load_weights('simpleRNN_3pred.h5')

In [64]:
model.save_weights('simpleRNN_7pred.h5')

In [65]:
model.load_weights('simpleRNN_7pred.h5')

#First convert input to index
#Expand dimension to match the model's output dimension
#Predict 8th character....can change this by changing pred_num
#Selecting the the character having maximum probability

In [79]:
def predict_next_char(inp):
    index = [char_to_index[i] for i in inp]
    arr = np.expand_dims(np.array(index), axis=0)
    prediction = model.predict(arr)
    return index_to_char[np.argmax(prediction)]

In [84]:
predict_next_char('can you')

'r'

# Predicting next word

In [86]:
ys = [[total_index[j+i] for j in range(1, len(total_index)-pred_num, pred_num)] for i in range(pred_num)]

In [88]:
Y_return = [np.stack(ys[i][:-2]) for i in range(pred_num)]

In [93]:
X_model = np.stack(X, 1)
Y_model = np.expand_dims(np.stack(Y_return, 1), axis=-1)

In [89]:
vocab_size = 86
n_fac = 42
hidden_layers = 256

In [95]:
word_model = Sequential([
        Embedding(vocab_size, n_fac, input_length=pred_num),
        SimpleRNN(hidden_layers, return_sequences=True, activation='relu'),
        TimeDistributed(Dense(vocab_size, activation='softmax'))
    ])

In [96]:
word_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 7, 42)             3612      
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, 7, 256)            76544     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 7, 86)             22102     
Total params: 102,258
Trainable params: 102,258
Non-trainable params: 0
_________________________________________________________________


In [97]:
word_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [98]:
word_model.fit(X_model, Y_model, batch_size=64, epochs=10)

Epoch 1/10
20749/20749 [==============================] - 3s 142us/step - loss: 3.0327
Epoch 2/10
20749/20749 [==============================] - 2s 119us/step - loss: 2.4111
Epoch 3/10
20749/20749 [==============================] - 2s 119us/step - loss: 2.2523
Epoch 4/10
20749/20749 [==============================] - 2s 119us/step - loss: 2.1543
Epoch 5/10
20749/20749 [==============================] - 2s 119us/step - loss: 2.0811
Epoch 6/10
20749/20749 [==============================] - 2s 116us/step - loss: 2.0251
Epoch 7/10
20749/20749 [==============================] - 2s 116us/step - loss: 1.9774
Epoch 8/10
20749/20749 [==============================] - 2s 115us/step - loss: 1.9369
Epoch 9/10
20749/20749 [==============================] - 2s 116us/step - loss: 1.9033
Epoch 10/10
20749/20749 [==============================] - 2s 116us/step - loss: 1.8709


In [104]:
return_model.optimizer.lr = 1e-4
return_model.fit(X_model, Y_model, batch_size=64, epochs=10)

Epoch 1/10
20749/20749 [==============================] - 2s 116us/step - loss: 1.7322
Epoch 2/10
20749/20749 [==============================] - 2s 114us/step - loss: 1.7141
Epoch 3/10
20749/20749 [==============================] - 2s 115us/step - loss: 1.6999
Epoch 4/10
20749/20749 [==============================] - 2s 115us/step - loss: 1.6849
Epoch 5/10
20749/20749 [==============================] - 2s 116us/step - loss: 1.6706
Epoch 6/10
20749/20749 [==============================] - 2s 116us/step - loss: 1.6574
Epoch 7/10
20749/20749 [==============================] - 2s 115us/step - loss: 1.6463
Epoch 8/10
20749/20749 [==============================] - 2s 115us/step - loss: 1.6338
Epoch 9/10
20749/20749 [==============================] - 2s 115us/step - loss: 1.6246
Epoch 10/10
20749/20749 [==============================] - 2s 114us/step - loss: 1.6141


In [105]:
def predict_every_char(inp):
    l = []
    p = 0
    while p<len(inp):
        pre_inp = inp[p:p+pred_num]
        if len(pre_inp) < pred_num:
            pre_inp = pre_inp + ' '*(pred_num - len(pre_inp))
            l.append(pre_inp)
        else:
            l.append(pre_inp) 
        p+=pred_num

    final = []
    for half in l:
        index = [char_to_index[i] for i in half]
        arr = np.expand_dims(index, axis=0)
        prediction = return_model.predict(arr)
        final.append(''.join([index_to_char[np.argmax(i)] for i in prediction[0]]))
    
    return ''.join(final)

In [106]:
predict_every_char('and the boy left')

'nd Che texstoa .mL   '

In [107]:
predict_every_char('this is')

'hen cs '

# LSTM

In [109]:
LSTM_model = Sequential([
        Embedding(vocab_size, n_fac, input_length=pred_num, batch_input_shape=(64, 7)),
        BatchNormalization(),
        LSTM(hidden_layers, activation='tanh', return_sequences=True, stateful=True),
        TimeDistributed(Dense(vocab_size, activation='softmax'))
    ])

In [110]:
LSTM_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [111]:
divide = len(X_model)//64*64

In [125]:
LSTM_model.fit(X_model[:divide], Y_model[:divide], batch_size=64, epochs=20, shuffle=False)

Epoch 1/20
20736/20736 [==============================] - 6s 288us/step - loss: 1.9103
Epoch 2/20
20736/20736 [==============================] - 6s 280us/step - loss: 1.8943
Epoch 3/20
20736/20736 [==============================] - 6s 284us/step - loss: 1.8792
Epoch 4/20
20736/20736 [==============================] - 6s 283us/step - loss: 1.8650
Epoch 5/20
20736/20736 [==============================] - 6s 284us/step - loss: 1.8515
Epoch 6/20
20736/20736 [==============================] - 6s 283us/step - loss: 1.8384
Epoch 7/20
20736/20736 [==============================] - 6s 283us/step - loss: 1.8259
Epoch 8/20
20736/20736 [==============================] - 6s 283us/step - loss: 1.8138
Epoch 9/20
20736/20736 [==============================] - 6s 284us/step - loss: 1.8023
Epoch 10/20
20736/20736 [==============================] - 6s 282us/step - loss: 1.7905
Epoch 11/20
20736/20736 [==============================] - 6s 282us/step - loss: 1.7794
Epoch 12/20
20736/20736 [================

In [126]:
def predict_every_char_LSTM(inp):
    l = []
    p = 0
    while p<len(inp):
        pre_inp = inp[p:p+pred_num]
        if len(pre_inp) < pred_num:
            pre_inp = pre_inp + ' '*(pred_num - len(pre_inp))
            l.append(pre_inp)
        else:
            l.append(pre_inp) 
        p+=pred_num
    
    final = []
    for half in l:
        index = [char_to_index[i] for i in half]
        arr = np.expand_dims(index, axis=0)
        arr = np.resize(arr, (64, 7))
        prediction = LSTM_model.predict(arr, batch_size=64)
        final.append(''.join([index_to_char[np.argmax(i)] for i in prediction[0]]))
    return ''.join(final)

In [127]:
predict_every_char_LSTM('this is')

' en wn '